In [36]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np
import pandas as pd
import conversion_df

# Récuperation du json et conversion en quelque chose d'utilisable

# Conversion en base de données panda et nettoyage

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

## Création de dummys


In [37]:
df = conversion_df.data_frame_pandas('donnees_citroen.json')

In [38]:
df.dtypes


marque                    object
modele                    object
carburant                 object
prix                       int64
kilometrage              float64
garantie_kilometrage      object
boite_de_vitesse          object
transmission             float64
couleur                   object
garantie                  object
date_mise_circulation      int64
puissance                float64
silhouette                object
nb_places                float64
utilisation_prec          object
puissance_fiscale        float64
critair                  float64
ptac                     float64
nb_portes                float64
dtype: object

In [39]:
df.ptac

0      1835.0
1      2300.0
2      1735.0
3      1540.0
4      1600.0
        ...  
595    1795.0
596    1610.0
597    1610.0
598    1610.0
599    1610.0
Name: ptac, Length: 600, dtype: float64

In [40]:
df.convert_dtypes().dtypes

marque                   string
modele                   string
carburant                string
prix                      Int64
kilometrage               Int64
garantie_kilometrage     string
boite_de_vitesse         string
transmission              Int64
couleur                  string
garantie                 string
date_mise_circulation     Int64
puissance                 Int64
silhouette               string
nb_places                 Int64
utilisation_prec         string
puissance_fiscale         Int64
critair                   Int64
ptac                      Int64
nb_portes                 Int64
dtype: object

## Passage sous numpy + ML

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from importlib import reload
import conversion_df
import pandas as pd

conversion_df = reload(conversion_df)

In [42]:
X, y = conversion_df.data_frame_modele()

In [43]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [44]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.812e+08, tolerance: 5.680e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.518e+09, tolerance: 5.525e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

In [100]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.015625, 'l1_ratio': 0.75}, 0.9174081996759759)

In [48]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [49]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [50]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001029,0.000643,0.049319,0.077393,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.515631,0.541902,0.381151,0.367331,0.520780,0.465359,0.075044,12
1,0.001004,0.000013,0.011752,0.000941,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.537089,0.570391,0.413332,0.423958,0.562600,0.501474,0.068603,3
2,0.000644,0.000439,0.010326,0.000995,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.524994,0.535793,0.392885,0.357530,0.537730,0.469786,0.078149,11
3,0.001105,0.000214,0.011623,0.001254,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.545500,0.568921,0.424024,0.421908,0.578683,0.507807,0.070110,1
4,0.001084,0.000132,0.009509,0.000779,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.529921,0.531394,0.391816,0.317182,0.537044,0.461471,0.090506,13


In [51]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 6, 'weights': 'distance'}, 0.5078072454925892)

In [52]:
from sklearn.ensemble import RandomForestRegressor

In [95]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256, 512, 750),
        "max_depth": (1,10,50,100,200),
        "min_samples_leaf": (1,2,5,10)
    }
)

In [96]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030182,0.007899,0.002880,0.000776,1,1,8,"{'max_depth': 1, 'min_samples_leaf': 1, 'n_est...",0.554134,0.507499,0.515860,0.531583,0.465724,0.514960,0.029297,145
1,0.040680,0.008152,0.002684,0.000777,1,1,16,"{'max_depth': 1, 'min_samples_leaf': 1, 'n_est...",0.531044,0.528201,0.519579,0.525254,0.461331,0.513082,0.026152,154
2,0.059940,0.004417,0.003443,0.000533,1,1,32,"{'max_depth': 1, 'min_samples_leaf': 1, 'n_est...",0.546700,0.528089,0.490790,0.533299,0.455734,0.510923,0.033246,157
3,0.132381,0.027062,0.006011,0.000543,1,1,64,"{'max_depth': 1, 'min_samples_leaf': 1, 'n_est...",0.545274,0.528306,0.494300,0.536892,0.461444,0.513243,0.031158,153
4,0.236248,0.011272,0.010360,0.000385,1,1,128,"{'max_depth': 1, 'min_samples_leaf': 1, 'n_est...",0.538922,0.528326,0.515373,0.535657,0.464601,0.516576,0.027219,133


In [97]:
rfr_gs.best_params_, rfr_gs.best_score_

({'max_depth': 200, 'min_samples_leaf': 1, 'n_estimators': 512},
 0.9087135607005093)

In [56]:
from sklearn.svm import SVR

In [111]:
svr = SVR(kernel='poly')
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "degree": (2,4,8,16)
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

In [ ]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 10}, 0.078272633012849)

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [60]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [61]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [62]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.045176,0.003580,0.023196,0.001874,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.011035,-0.000980,-0.004867,-0.016168,-0.060419,-0.018694,0.021499,8
1,0.042613,0.000380,0.022075,0.000116,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.011040,-0.000980,-0.004875,-0.016168,-0.060406,-0.018694,0.021492,9
2,0.042340,0.000520,0.021676,0.000353,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.011257,-0.000910,-0.005022,-0.016168,-0.060079,-0.018687,0.021343,7
3,0.042304,0.000146,0.022104,0.000386,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.007116,0.003547,-0.000874,-0.011065,-0.054989,-0.014099,0.021054,4
4,0.042158,0.000498,0.022490,0.000242,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.007136,0.003546,-0.000874,-0.011065,-0.054990,-0.014104,0.021052,5


In [63]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 0.02991012108293569)

In [64]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [65]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [66]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [67]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.472130,0.086043,0.002666,0.000938,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-7.226165,-6.153329,-7.437686,-8.207418,-6.755187,-7.155957,0.686404,24
1,2.714988,0.162071,0.002360,0.000814,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-7.057060,-5.968051,-7.176973,-7.902737,-6.649349,-6.950834,0.636443,18
2,4.381803,0.139657,0.003925,0.000132,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-6.564133,-5.704283,-6.844026,-7.426439,-6.175984,-6.542973,0.584752,11
3,1.407072,0.326447,0.001183,0.000407,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-2.473767,-2.330474,-3.208376,-4.529774,-2.010705,-2.910619,0.899730,3
4,1.860738,0.417706,0.003682,0.000946,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-7.159940,-6.212756,-7.460333,-8.067242,-6.750973,-7.130249,0.628241,19


In [68]:
pln_gs.best_params_

pln_gs.best_score_

-2.5316190287999376

In [69]:
pln_gs.best_params_

{'neurones__alpha': 0.001, 'neurones__hidden_layer_sizes': (20, 20)}

In [70]:
essai = en_gs.best_estimator_

In [71]:
X

array([[7.5967e+04, 2.0190e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.9066e+04, 2.0210e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.1997e+04, 2.0210e+03, 1.3000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.0000e+01, 2.0220e+03, 1.1000e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [72]:
prix_prédits = essai.predict(X)

In [73]:
prix_reels = y

In [74]:
prix_reels,prix_prédits

(array([20290, 31900, 33920, ..., 22689, 22689, 22689], dtype=int64),
 array([21747.1039106 , 29540.97805825, 30080.59216952, ...,
        22953.44940605, 23197.92601548, 23197.92601548]))

In [75]:
len(prix_prédits)

1200

In [76]:
len(prix_reels)

1200

In [77]:
prix_prédits

array([21747.1039106 , 29540.97805825, 30080.59216952, ...,
       22953.44940605, 23197.92601548, 23197.92601548])

In [84]:
en_gs.score(X_te, y_te)

0.9269994803810756

In [105]:
rfr_gs.score(X_te, y_te)

0.932590354508677

ValueError: could not convert string to float: 'Citroen'

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus
